<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Using OpenMDAO

In this notebook, we will present a basic introduction to OpenMDAO and how to "translate" basic Python into this framework. More details on OpenMDAO as well as tutorials can be found [here](https://openmdao.org/newdocs/versions/latest/main.html).

This tutorial should not be seen as detailed tutorial for OpenMDAO but rather an illustrated explanation of its basic features. 

## OpenMDAO for beginners

OpenMDAO uses components as a building block to solve any problem. A component is a simple way to implement a numerical model, a sizing method or any other computation that can be used in a sizing process. A very practical way of visualizing one is to see it as a transformation box that takes some inputs does calculations and then returns outputs. Those components can then be gathered in groups, of various size and composition, which can then be used to define a model for the problem we want to solve.

<div class="row">
  <div class="column">
    <img src="./img/Component.png" width="600">
  </div>
</div>

OpenMDAO provides 3 main types of [component](https://openmdao.org/newdocs/versions/latest/features/building_blocks/components/components.html) :
* [IndepVarComp](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/indepvarcomp.html) which defines independent variables.
* [ExplicitComponent](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/explicit_component.html) which defines dependent variables that are computed explicitly, meaning there is a direct link between the output(s) and the input(s) (see the example above).
* [ImplicitComponent](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/implicit_component.html) which defines dependent variables that are computed implicitly, meaning the variables are linked through an implicit relation. An example from OpenMDAO documentation is a component that finds y such that cos(x*y)-z*y=0.

<div class="alert alert-block alert-info">
<b>Remark</b>
    
Among the more specific [components](https://openmdao.org/newdocs/versions/latest/features/building_blocks/components/components.html) OpenMDAO provides, the [ExternalCodeComp](https://openmdao.org/newdocs/versions/latest/features/building_blocks/components/external_code_comp.html) is a specialized ExplicitComponent which allows to use any external software that can be run through a script. For instance in FAST-OAD_CS25 and FAST-OAD-GA, this type of component is used to run [Xfoil](https://web.mit.edu/drela/Public/web/xfoil/). This is however more suited to advanced user. 
</div>

For ExplicitComponent and ImplicitComponent, the most common ones, the structure will always remain the same and will be explained below. In order to illustrate the syntax we will walk through the process of translating one of the pure_python base brick as an OpenMDAO component. We will then see how to assemble those components as groups to later solve the problem using OpenMDAO. When possible, the parallel will be made with the pure_python definition of the component. 

For this exercice, we chose the component that computes the lift-to-drag ratio in cruise.

<div class="row">
  <div class="column">
    <img src="./img/Lift_to_drag.png" width="400">
  </div>
</div>

### Defining the component

Components are defined as Python classes which must inherit from the OpenMDAO type of component we want to use for our method. In our case, as in a vast majority of models in FAST-OAD plugins, the component is an ExplicitComponent. Indeed, the goal of this component is simply to compute the lift-to-drag ratio in cruise. Since we know how to explicitly link the inputs to the output, this will be an ExplicitComponent. Thus, we must import the component from OpenMDAO API and create a new class based on the ExplicitComponent.

> ``` python
>import openmdao.api as om
>
>
>class ComputeLiftToDragRatio(om.ExplicitComponent):
>    """
>    Computes the lift to drag ratio considering a lift equilibrium in cruise and a simple quadratic model
>    """
> ```

### Defining the inputs/outputs

Now that we have defined the component, we must define its inputs and outputs. Contrary to a classic Python function, where they are defined in the signature, the definition is made in a separate class function called `setup`. 

For each input/output we want to add to our component, we must provide :
* The name of the variable: There is in OpenMDAO two variable linking startegies: promotion vs connection. In the former, data are linked based on their names while in the latter, the connection have to be made by hand. Because promoting variables is most convenient, this is the strategy that has been retained here and in most of FAST-OAD plugins. Consequently, it is crucial to keep the naming consistent. For instance two inputs/outputs named aspect_ratio and wing_aspect_ratio will not represent the same quantity even if, in reality, they might mean the same thing. For future users, a more detailed explanation of the variable naming in FAST-OAD is available [here](https://fast-oad.readthedocs.io/en/latest/documentation/custom_modules/add_modules.html?highlight=variable%20naming#variable-naming).
* The unit of the variable: OpenMDAO supports automatic unit conversion. Therefore, different components that work with a same variable may use its value in different units without having to explicitly do the conversion.
* The shape of the variable: By default, OpenMDAO variables are (1,)-shaped numpy arrays. They can have a different shape, which shall be declared in `setup`. Yet, an [advanced feature](https://openmdao.org/newdocs/versions/latest/features/experimental/dyn_shapes.html?highlight=shape_by_conn) allows a variable shape to be defined at runtime, based on a shape defined in another component.
* The default value of the variable: This is the value OpenMDAO will use if the variable hasn't been computed beforehand. It may be a quick way of providing input values, but there are more versatile solutions, so we generally use `numpy.nan`.

As a reminder, the way we defined inputs in pure python was via the function signature. As for the handling of the units, it was made based on the assumption that each time we called the function we would respect the units in the docstring. The outputs were defined via the return instruction.

> ``` python
>def compute_l_d(cruise_altitude, cruise_speed, cd0, k, mtow, wing_area):
>    """
>    Computes the lift to drag ratio considering a lift equilibrium in cruise and a simple quadratic model
>
>    :param cruise_altitude: Cruise altitude, in m
>    :param cruise_speed: Cruise speed, in m/s
>    :param cd0: Profile drag, no unit
>    :param k: Lift induced drag coefficient, no unit
>    :param mtow: Max Take-Off Weight, in kg
>    :param wing_area: Wing area, in m2
>
>    :return l_d: Lift-to-drag ratio in cruise conditions, no unit
>    """
> ```

...

> ``` python
> return l_d
> ```

The same thing is done in OpenMDAO by overwriting the `setup` function and defining the inputs/outputs specific to the component. In our case, it will be :

> ``` python
>def setup(self):
>
>    # Defining the input(s)
>
>    self.add_input(name="cruise_altitude", units="m", val=np.nan)  # For a float or int, shape don't have to be
>    # provided
>    self.add_input(name="cruise_speed", units="m/s", val=np.nan)
>    self.add_input(name="profile_drag_coefficient", val=np.nan)  # When the quantity does not have a unit, the units
>    # field doesn't need to be filled
>    self.add_input(name="induced_drag_coefficient", val=np.nan)
>    self.add_input(name="mtow", units="kg", val=np.nan)
>    self.add_input(name="wing_area", units="m**2", val=np.nan)  # OpenMDAO understands the multiplication/division
>    # of units in between them hence why the m**2 is understood
>
>    # Defining the output(s)
>
>    self.add_output(name="l_d_ratio")
> ```

<div class="alert alert-block alert-info">
<b>Remark</b>

When not specified, the default value for any input/output is set to 1.0 by OpenMDAO. It can be overwritten by filling the val option when adding an input/output. In FAST-OAD, we tend to overwrite the default value for **inputs** with a `numpy.nan`. This way we can make sure the process purposedly crash when the default value for an input is used. This enables to track if an input has not been set in data file nor in other components.

For **outputs** however, a `numpy.nan` value should not be used because OpenMDAO will base its residuals computation on it. If a `numpy.nan` value is used on one of the outputs we loop on, OpenMDAO will see a residual equals to `numpy.nan`, causing the process to crash. The default value can however be overwritten by something else, in which case this value will be used as the value in the first iteration. It can also be defined in OpenMDAO once the problem is setup, which is how we will do it here. In FAST-OAD however, it is not possible to define it afterwards so it must be done when declaring the variable as output.
</div>

<div class="alert alert-block alert-info">

<b>Remark (for advanced users)</b>
    
OpenMDAO allows to define options for each component. An oversimplification of the options would be to see them as "super-inputs". They can be used inside the setup to, for instance, change the inputs based on the context. For example, if we wanted to have the choice to use either cruise conditions or low speed conditions, we could have set a low_speed option depending on which we could have defined the cruise conditions or the low speed conditions as inputs. The definition of the option is done in the `initialize` function which must overwrite. We will then be able to call it into the `setup`. This is shown below but not implemented in the modules. Comments were not rewritten.
   
``` python
def initialize(self):
    # Defining the option, with name, default value and Python type (float, int, tuple, bool, ...)
    self.options.declare("low_speed", default="True", types=bool)


def setup(self):

    # Defining the input(s)
    
    # If we are in low speed conditions we add the low speed speed and and low speed altitude
    if self.options["low_speed"]:
        self.add_input(name="low_speed_altitude", units="m", val=np.nan)
        self.add_input(name="low_speed_speed", units="m/s", val=np.nan)
    else:
        self.add_input(name="cruise_altitude", units="m", val=np.nan)
        self.add_input(name="cruise_speed", units="m/s", val=np.nan)
    self.add_input(name="profile_drag_coefficient", val=np.nan)
    self.add_input(name="induced_drag_coefficient", val=np.nan)
    self.add_input(name="mtow", units="kg", val=np.nan)
    self.add_input(name="wing_area", units="m**2", val=np.nan)

    # Defining the output(s)

    self.add_output(name="l_d_ratio")
```
</div>

### Linking the outputs/inputs

Now that we have defined which input variables are going to be used and what outputs will be returned, it is time to link them. This is done in the `compute` function which signature will always be the same, only the content will change. One less thing to worry about :). This was previously done inside the function we defined for each computation, like so:

> ``` python
>def compute_l_d(cruise_altitude, cruise_speed, cd0, k, mtow, wing_area):
>
>    # Free-fall acceleration constant
>    g = 9.81
>  
>    # Air density at sea-level, to compute it, we will use the Atmosphere model available in FAST-OAD, so we will create
>    # an Atmosphere instance using the cruise altitude and extract its density attribute
>    atm = Atmosphere(altitude=cruise_altitude, altitude_in_feet=False)
>    rho = atm.density
>  
>    # Computation of the cruise lift coefficient using a simple equilibrium
>    cl = (mtow * g) / (0.5 * rho * cruise_speed ** 2.0 * wing_area)
>  
>    # Computation of the cruise drag coefficient using the simple quadratic model
>    cd = cd0 + k * cl ** 2
>  
>    # Computation of the ratio
>    l_d = cl / cd
>  
>    return l_d
> ```

<div class="alert alert-block alert-info">
<b>Remark:</b>

Before continuing it is important to know that the `inputs` and `outputs` arguments in OpenMDAO are similar to dictionaries. Dictionary keys are variable names and values are either taken from previous components, if names match, or are taken as the default value. Consequently, in order for our component to actually modify the value of the output we declared, we must change the value in the outputs dictionary. For instance, to retrieve the <b>MTOW</b>, we will do:

``` python
inputs["mtow"]
```

It can also be used as such in the computation, meaning we could do:

``` python
c = inputs["mtow"] * inputs["wing_area"]
```

However we will first assign it to a local variable to make the rest of the computation clearer. This would turn the line above into: 

``` python
# Assigning the input to local variable for clarity
mtow = inputs["mtow"]
wing_area = inputs["wing_area"]

# Actual computation
c = mtow * wing_area
```
</div>

All in all, the translation of the component that computes the lift-to-drag ratio in the OpenMDAO format will look like:

> ``` python
> def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):
>
>     # Assigning the input to local variable for clarity
>     cruise_altitude = inputs["cruise_altitude"]
>     cruise_speed = inputs["cruise_speed"]
>     cd0 = inputs["profile_drag_coefficient"]
>     k = inputs["induced_drag_coefficient"]
>     mtow = inputs["mtow"]
>     wing_area = inputs["wing_area"]
>    
>     # Free-fall acceleration constant
>     g = 9.81
>    
>     # Air density at sea-level, to compute it, we will use the Atmosphere
>     # model available in FAST-OAD, so we will create an Atmosphere instance
>     # using the cruise altitude and extract its density attribute
>     atm = Atmosphere(altitude=cruise_altitude, altitude_in_feet=False)
>     rho = atm.density
>    
>     # Computation of the cruise lift coefficient using a simple equilibrium
>     cl = (mtow * g) / (0.5 * rho * cruise_speed ** 2.0 * wing_area)
>    
>     # Computation of the cruise drag coefficient using the simple quadratic model
>     cd = cd0 + k * cl ** 2
>    
>     # Computation of the ratio
>     l_d = cl / cd
>    
>     outputs["l_d_ratio"] = l_d
> ```

The reader will notice that, outside of the way inputs are retrieved and outputs are assigned, it is strictly the same code. The actual implementation of this in a .py file is available [here](modules/OpenMDAO/aerodynamics/sub_components/compute_lift_to_drag_ratio.py).

The rest of the implementation of the base bricks of the code under the OpenMDAO framework is done in separate folders for each discipline, similarly to what was done in pure Python. As such :

The **geometry** folder (modules/OpenMDAO/geometry) contains:
* [The wing area computation](modules/OpenMDAO/geometry/sub_components/compute_wing_area.py)

The **aerodynamics** folder (modules/OpenMDAO/aerodynamics) contains:
* [The profile drag computation](modules/OpenMDAO/aerodynamics/sub_components/compute_profile_drag.py)
* [The induced drag coefficient computation](modules/OpenMDAO/aerodynamics/sub_components/compute_induced_drag_coefficient.py)
* [The cruise lift-to-drag ratio computation](modules/OpenMDAO/aerodynamics/sub_components/compute_lift_to_drag_ratio.py)

The **mass** folder (modules/OpenMDAO/mass) contains:
* [The wing mass computation](modules/OpenMDAO/mass/sub_components/compute_wing_mass.py)
* [A complete OWE computation](modules/OpenMDAO/mass/sub_components/compute_owe.py)
* [A incomplete OWE computation](modules/OpenMDAO/mass/sub_components/compute_owe_exercise.py)

The **performance** (modules/OpenMDAO/performance) folder contains:
* [The mission fuel computation](modules/OpenMDAO/performance/sub_components/compute_fuel_mass.py)

The **update_mtow** (modules/OpenMDAO/update_mtow) folder contains:
* [The MTOW computation](modules/OpenMDAO/update_mtow/update_mtow.py)

<div class="alert alert-block alert-danger">
The reader will notice that there are two OWE computations and that in some files, some components are written twice. The reason for that is that the choice was left to the user to either write the OWE component as an exercise or to use a fully functionning one. The OWE computation to complete is located in the <a href="modules/OpenMDAO/mass/sub_components/compute_owe_exercise.py">compute_owe_exercise.py file</a>. When filling it, keep in mind to not change the names compared to previous and future modules and be careful with units. The choice of which components to use can be made by executing the next cell and choosing the appropriate option. In case of a change of mind don't forget to rerun all cells <b>after</b> the widget.
</div>

In [ ]:
from modules.widget import ChoiceForOWE

choice = ChoiceForOWE()

## Groups

Now that we have defined the base brick of our components, it is time to assemble them into the MDA we want to solve with OpenMDAO. As in the original problem, we will first assemble them by sub-categories and then assemble them in a global model.

Assembling components in OpenMDAO is done using a [Group](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_groups/main.html) class. Groups can have multiple purposes in OpenMDAO: assembling components (for convenience, which is what we will do), hiding variables and declaring connections (in case the choice was made to connect variables by hand) or defining sub-loops/sub-solver. As for the other types of components, options can be defined for Groups.

The Group component also has a `setup` function, however, instead of declaring its inputs and outputs, we declare the base bricks/components we want to add. A Group can contains explicit components, implicit components, independant variable components, external code components and even other group.

It may happen that, when setting up a group, the output(s) of one component of the group is(are) linked to the input(s) of a previous one (or to one of a previous group). This is will be the case for the MTOW in our example. When this occurs, a solver need to be added to enable the iterative solving of this loop. Without this solver, and even if OpenMDAO can automatically detect the variables that are looped on, the group will simply be executed once, without assurance that the entry of the loop is equal to its exit.

The syntax used to add a component inside a group is via the `add_subsystem` functions in which we must fill :
* A name for the subsystem we are adding. Using the chosen name, the added subsystem will be reachable as a Python attribute of the parent group. Thus, it has to be Python-compliant. In addition to that, when OpenMDAO returns an error, the path to the faulty component is displayed using these names. Making the name unique in the problem can thus help debugging.
* An instance of the subsystem we want to add, which means we need to first import all of the subsystem we want to add.
* What variable we want to promote, or in simpler terms, what variable we want the rest of the groups/model to see. As ex plained before we use a promotion strategy, hence, we will use the "\*" symbol meaning we promote all variables.

For instance, let's build the aerodynamics group. As in the original problem, we want to add the computation of the profile drag, the induced drag coefficient and the lift to drag ratio. Let's start by defining the Group, giving it a name and importing all the components we want to add. This is done like so:

> ``` python
> import openmdao.api as om
>
> from .sub_components.compute_profile_drag import ComputeProfileDrag
> from .sub_components.compute_induced_drag_coefficient import ComputeInducedDragCoefficient
> from .sub_components.compute_lift_to_drag_ratio import ComputeLiftToDragRatio
>
>
> class ComputeAerodynamics(om.Group):
>```

Then we can call the `setup` function to add the subsystems abd promote all variables to make them usable by the rest of the components. This is done like so:

> ``` python
> def setup(self):
>
>     self.add_subsystem(name="compute_profile_drag", subsys=ComputeProfileDrag(), promotes=["*"])
>     self.add_subsystem(
>         name="compute_induced_drag", subsys=ComputeInducedDragCoefficient(), promotes=["*"]
>     )
>     self.add_subsystem(
>         name="compute_lift_to_drag", subsys=ComputeLiftToDragRatio(), promotes=["*"]
>     )
>```

The group in full is written in the [aerodynamics file](modules/OpenMDAO/aerodynamics/aerodynamics.py)

<div class="alert alert-block alert-info">
<b>Remark:</b>
    
If we had a component that had options, we would have needed to fill them as if they were the inputs of the class constructor when instantiating said component in the `add_subsytem` function. If we retake the example of the ComputeLiftToDragRatio component that we define with an option earlier in the notebook, it means we should have written:

``` python
    def setup(self):
        
        self.add_subsystem(
            name="compute_lift_to_drag", subsys=ComputeLiftToDragRatio(low_speed=False), promotes=["*"]
        )
```
</div>

The creation of the rest of the groups will not be shown here. 

Now that every group corresponding to their discipline was created, we can create the global group that will gather the rest of them in what is called in the OpenMDAO framework a model. This is done the same way as for the discipline groups. The order in which they are added however matters, as was the case for the code in pure Python. We chose to keep the same order.

> ``` python
> from .geometry.geometry import ComputeGeometry
> from .aerodynamics.aerodynamics import ComputeAerodynamics
> from .mass.mass import ComputeMass
> from .performance.performance import ComputeNewMtow
> 
> import openmdao.api as om
> 
> 
> class UpdateMTOW(om.Group):
>     """
>     Gather all the discipline module/groups into the main problem
>     """
> 
>     def setup(self):
> 
>         self.add_subsystem(name="compute_geometry", subsys=ComputeGeometry(), promotes=["*"])
>         self.add_subsystem(name="compute_aerodynamics", subsys=ComputeAerodynamics(), promotes=["*"])
>         self.add_subsystem(name="compute_mass", subsys=ComputeMass(), promotes=["*"])
>         self.add_subsystem(name="compute_performance", subsys=ComputePerformance(), promotes=["*"])
>```

## Setting up the problem

Now that we have created the model that we want to solve, we can setup and run the problem, which is how we tell OpenMDAO to solve the problem based on the aircraft characteristics we give it.

First thing we have to do is to give to OpenMDAO the characteristics of the aircraft. There are two ways of doing it :
* After adding the model to the problem, we can set the value of the aircraft characteristics by using the `prob.set_val()` function for each input.
* The other way of doing it is by creating an IndepVarComp with the all the aircraft characteristics. Because this IndepVarComp will be its own component, we will be able to reuse it for other OpenMDAO problems on the same aircraft, which is not the cas with `prob.set_val()`.

In [ ]:
import openmdao.api as om

# Let's start by creating the IVC (IndepVarComp) and add the data related to ...
input_ivc = om.IndepVarComp()

# ... the geometry
input_ivc.add_output(name="wing_loading", val=115.0, units="kg/m**2")
input_ivc.add_output(name="aspect_ratio", val=10.0)

# ... the target mission
input_ivc.add_output(name="cruise_altitude", val=2500.0, units="m")
input_ivc.add_output(name="cruise_speed", val=80.0, units="m/s")
input_ivc.add_output(name="mission_range", val=750, units="NM")
# Notice how we give the value of the range in nautical mile, which is much more clear for anyone in the aviation business.
# This is allowed by OpenMDAO.
input_ivc.add_output(name="payload", val=320, units="kg")

# ... the propulsion technology
input_ivc.add_output(name="tsfc", val=7.3e-6, units="kg/N/s")

Now that we have filled the inputs that we are going to use, we can create a problem, give him the model we want him to solve, the associated input and set it up. For the sake of showing the second way to set the inputs of the problem, we will create a second one with the same model and not add the IndepVarComp.

In [ ]:
from modules.OpenMDAO.mtow_loop import SizingLoopMTOWCorrect, SizingLoopMTOWExercise

# Create two problems
prob = om.Problem()
prob_without_ivc = om.Problem()

# Adding the inputs, but not for the problem without_ivc
prob.model.add_subsystem(name="inputs_definition", subsys=input_ivc, promotes=["*"])

# Adding the actual problem based on user choice
if choice.choice == "correct":
    prob.model.add_subsystem(name="mtow_solving", subsys=SizingLoopMTOWCorrect(), promotes=["*"])
    prob_without_ivc.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWCorrect(), promotes=["*"]
    )
else:
    prob.model.add_subsystem(name="mtow_solving", subsys=SizingLoopMTOWExercise(), promotes=["*"])
    prob_without_ivc.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWExercise(), promotes=["*"]
    )

# Adding the solvers for the problem.
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.DirectSolver()

prob_without_ivc.model.nonlinear_solver = om.NonlinearBlockGS()
prob_without_ivc.model.linear_solver = om.DirectSolver()

# Print level can be set at greater values to show more information about the convergence of the process
prob.model.set_solver_print(level=1)
prob_without_ivc.model.set_solver_print(level=1)

prob.model.nonlinear_solver.options["maxiter"] = 50
prob.model.nonlinear_solver.options["atol"] = 1e-4

prob_without_ivc.model.nonlinear_solver.options["maxiter"] = 50
prob_without_ivc.model.nonlinear_solver.options["atol"] = 1e-4

# We can also change the properties of the solver, such as the tolerance at which we solve the problem or the maximum
# number of iteration after which the process stops

# Setting up the problem
prob.setup()
prob_without_ivc.setup()

At this point in the code, we would be able to run the problem **with** IVC if it was setup correctly. The problem **without** IVC, as for itself, will try to use the default value if ran as is. Before running the problem, however, we will present a useful tool implemented in OpenMDAO called the N2 chart.

## [N2 chart](https://en.wikipedia.org/wiki/N2_chart)

Quoting [OpenMDAO's documentation](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_basics/n2_basics.html?highlight=n2%20chart): an N2 diagram, also known as an N-squared diagram, is a diagram in the shape of a matrix, representing functional or physical interfaces between system elements. In essence, this diagram contains the name of the subsystems (the provided name in the `add_subsystem` command), the name of the variables as well as the interaction between subsystems. The squares in the diagonal represent the modules, the ones on the upper-right half represent a direct transmission between two modules and the ones in the lower-left half represent variables that causes a loop. It can also be very useful to spot missing problem inputs. Said entries will be connected to an OpenMDAO component called `Auto-IVC` which means that their default value will be used, making them easily identifiable. 

As an example, we will draw the N2 diagram for our problem **without** IVC and we will see that a lot of values will be connected to `Auto-IVC`. This is done in the following cell.

In [ ]:
om.n2(prob_without_ivc)

Consequently it can also be used to spot missing component or miscoded one. In such case, when plotting the N2 diagram, we would see a value that we did not expect among problem inputs inside the `Auto-IVC`. For instance, in the case of the problem with the component left as exercise we would see the following N2 diagram:

<div class="row">
  <div class="column">
    <img src="./img/N2_diagram.png" width="1000">
  </div>
</div>

Since we expect to compute the OWE it should not appear in the `Auto-IVC` which means we have a bug.

On the other hand, if the computation was filled in properly (in terms of syntax, OpenMDAO won't know if an error was made in the formula, we still need a bit of engineering knowledge after all), it will look like this:

<div class="row">
  <div class="column">
    <img src="./img/N2_diagram_correct.png" width="1000">
  </div>
</div>

You will easily notice that the OWE is no longer highlighted in orange, meaning it is now properly connected between `compute_fuel_mass` and `compute_performance`.

## Running the problem

Final step before running both problems, we need to set the value for `prob_without_ivc`. This is done in the following cell thanks to the `prob.set_val()` function.

In [ ]:
# Let's add the data related to ...
# ... the geometry
prob_without_ivc.set_val(name="wing_loading", val=115.0, units="kg/m**2")
prob_without_ivc.set_val(name="aspect_ratio", val=10.0)

# ... the target mission
prob_without_ivc.set_val(name="cruise_altitude", val=2500.0, units="m")
prob_without_ivc.set_val(name="cruise_speed", val=80.0, units="m/s")
prob_without_ivc.set_val(name="mission_range", val=750, units="NM")
# Same remark, we can now give the range in nautical mile
prob_without_ivc.set_val(name="payload", val=320, units="kg")

# ... the propulsion technology
prob_without_ivc.set_val(name="tsfc", val=7.3e-6, units="kg/N/s")

Now, we can run the model by simply using the `run_model` command. Let's start by doing the problem with the IVC:

In [ ]:
import numpy as np

# And finally running the problem !
prob.run_model()

print(
    "The solution MTOW for the problem with IVC is :",
    float(np.round(prob.get_val("mtow", units="kg"), 1)),
    "kg",
)

Here, the problem did not converged. The reason behind that is the fact that we did not assign an initial value for the **MTOW**. Because of that, OpenMDAO took the default value which is 1.0. The direct consequence is that the second value that the **MTOW** took was greater than 1e+52 kg leading to a vicious circle of ever greater mass that culminated in a non-convergence of the OpenMDAO problem. A simple solution to solve this issue is to provide an educated first guess. This can be done using the `prob.set_val()` function.

In [ ]:
# Provide a first guess that is not too far from reality
prob.setup()
prob.set_val(name="mtow", val=500.0, units="kg")

# Rerun the problem
prob.run_model()

print(
    "The solution MTOW for the problem with IVC is :",
    float(np.round(prob.get_val("mtow", units="kg"), 1)),
    "kg",
)

prob_without_ivc.set_val(name="mtow", val=500.0, units="kg")
prob_without_ivc.run_model()

print(
    "The solution MTOW for the problem without IVC is :",
    float(np.round(prob_without_ivc.get_val("mtow", units="kg"), 1)),
    "kg",
)

Here we can see that both our problem converged on the same value. We get an identical result to what we obtained in pure Python. As a reminder, we found, for the same input parameters, a **MTOW** of around 1065.8 kg. In addition to the automatic unit conversion which was identified as a weak point from Python, here, we can access any intermediate variable. For instance, if we want the fuel consumed for the mission or the mass of the wing we just need to do the following:

In [ ]:
print(
    "Fuel consumed during the mission:",
    float(np.round(prob.get_val("mission_fuel", units="kg"), 1)),
    "kg",
)
print("Area of the wing:", float(np.round(prob.get_val("wing_area", units="m**2"), 1)), "m**2")

We can also use this functionality to check the coherency of the results, for instance according to [1], the wing mass represents around 10% of the **MTOW**. In our case, we have:

In [ ]:
print("Mass of the wing:", float(np.round(prob.get_val("wing_mass", units="kg"), 1)), "kg")
print("MTOW:", float(np.round(prob.get_val("mtow", units="kg"), 1)), "kg")

This seems to confirm that the formula was correctly implemented.

## Running the MDO

As we did in the previous two examples, we are now going to run an MDO on our model to try to find the aspect ratio that gives the best **MTOW**. We will then run a new MDO to find the best fuel consumption during the mission. Once this is done, and as in the previous notebooks, we will try to find the best combination of cruise speed and aspect ratio in order to, again, minimize the fuel consumption.

### Finding the best MTOW

The setup of the optimization problem is very similar to what was done before: first setup the inputs and then the optimization model. However here, in addition to that, we need to define the objectives and design variables of our problem. The aspect ratio, previously an input, will now be a design variable. We can still reuse the previous IVC for the inputs as the declaration of the aspect ratio as a design variable will allow the optimizer to change it.

In [ ]:
# We define the problem
prob_optim = om.Problem()

# Adding the inputs
prob_optim.model.add_subsystem(name="inputs_definition", subsys=input_ivc, promotes=["*"])

# Adding the actual problem based on user choice
if choice.choice == "correct":
    prob_optim.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWCorrect(), promotes=["*"]
    )
else:
    prob_optim.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWExercise(), promotes=["*"]
    )

# Add the solver for the MTOW loop and tune them to fit what we did in the MDA
prob_optim.model.nonlinear_solver = om.NonlinearBlockGS()
prob_optim.model.linear_solver = om.DirectSolver()

prob_optim.model.set_solver_print(level=1)

prob_optim.model.nonlinear_solver.options["maxiter"] = 50
prob_optim.model.nonlinear_solver.options["atol"] = 1e-4

# Add the optimization driver (can be copied as is, fine tuning of the optimizer is for advanced users)
prob_optim.driver = om.ScipyOptimizeDriver()
prob_optim.driver.options["optimizer"] = "COBYLA"
# prob.driver.options['maxiter'] = 100
prob_optim.driver.options["tol"] = 1e-4

# We define the aspect ratio as a design variable and we constrain it
prob_optim.model.add_design_var("aspect_ratio", lower=1.0, upper=20.0)
prob_optim.model.add_objective("mtow")

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer (can be copied as is)
prob_optim.model.approx_totals()

# Setting up the problem
prob_optim.setup()

# Initialization of the design var(s) and giving the mtow its initial value. It is important to initialize at a decent
# value or the optimizer might not converge
prob_optim.set_val(name="mtow", val=500.0, units="kg")
prob_optim.set_val(name="aspect_ratio", val=10.0)

# Runnin the optimization
prob_optim.run_driver()

# Printing the output
print("minimum found at")
print(float(np.round(prob_optim.get_val("aspect_ratio"), 4)))

print("minimum objective")
print(float(np.round(prob_optim.get_val("mtow", units="kg"), 4)), "kg")

We can see here that we have a close but smaller result than what we found in [the second notebook](02_pure_Python.ipynb). Indeed, the optimum **MTOW** was previously found equal to 999.114 kg for an aspect ratio of 3.495. Besides the improved accuracy, we didn't have to rewrite the models for the MDO, we only had to change how they were used. The same will go for the optimization of the fuel consumption in which we will just have to change the objective from "mtow" to "mission_fuel".

### Finding the best fuel consumption

Changing the optimization objective is done in the following cell where we just copy the lines from the previous cell, change the name of the problem to avoid any conflict and change the objective. Again, we won't have to change the input IVC as the design variable will not change and neither will the problem inputs. 

In [ ]:
# We define the problem
prob_optim_fuel = om.Problem()

# Adding the inputs
prob_optim_fuel.model.add_subsystem(name="inputs_definition", subsys=input_ivc, promotes=["*"])

# Adding the actual problem based on user choice
if choice.choice == "correct":
    prob_optim_fuel.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWCorrect(), promotes=["*"]
    )
else:
    prob_optim_fuel.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWExercise(), promotes=["*"]
    )

# Add the solver for the MTOW loop and tune them to fit what we did in the MDA
prob_optim_fuel.model.nonlinear_solver = om.NonlinearBlockGS()
prob_optim_fuel.model.linear_solver = om.DirectSolver()

prob_optim_fuel.model.set_solver_print(level=1)

prob_optim_fuel.model.nonlinear_solver.options["maxiter"] = 50
prob_optim_fuel.model.nonlinear_solver.options["atol"] = 1e-4

# Add the optimization driver (can be copied as is, fine tuning of the solver is for advanced users)
prob_optim_fuel.driver = om.ScipyOptimizeDriver()
prob_optim_fuel.driver.options["optimizer"] = "COBYLA"
# prob.driver.options['maxiter'] = 100
prob_optim_fuel.driver.options["tol"] = 1e-4

# We define the aspect ratio as a design variables and we constrain it
prob_optim_fuel.model.add_design_var("aspect_ratio", lower=1.0, upper=20.0)
prob_optim_fuel.model.add_objective("mission_fuel")

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob_optim_fuel.model.approx_totals()

# Setting up the problem
prob_optim_fuel.setup()

# Initialization of the design var(s) and giving the mtow its initial value. It is important to initialize at a decent
# value or the optimizer might not converge
prob_optim_fuel.set_val(name="mtow", val=500.0, units="kg")
prob_optim_fuel.set_val(name="aspect_ratio", val=10.0)

# Runnin the optimization
prob_optim_fuel.run_driver()

# Printing the output
print("minimum found at")
print(float(np.round(prob_optim_fuel.get_val("aspect_ratio"), 4)))

print("minimum objective")
print(float(np.round(prob_optim_fuel.get_val("mission_fuel", units="kg")[0], 4)), "kg")

Then again, our result is very close to what we found in [the second notebook](02_pure_Python.ipynb) (slightly better, actually).

<div class="alert alert-block alert-info">
<b>Remark:</b>

By default, the optimization in OpenMDAO is a minimization. It is however possible, when <a href=https://openmdao.org/newdocs/versions/latest/features/core_features/adding_desvars_cons_objs/adding_objective.html> declaring the objective</a>, to define a scaler. A common usage is to set the scaler to -1 to transform the minimization in a maximization.
    
</div>

### Two variables optimization

Let's now move on to the two variables optimization. The syntax is nearly identical to what we had in the previous optimizations, only this time we will define two design variables.

In [ ]:
# We define the problem
prob_optim_v3 = om.Problem()

# Adding the inputs
prob_optim_v3.model.add_subsystem(name="inputs_definition", subsys=input_ivc, promotes=["*"])

# Adding the actual problem based on user choice
if choice.choice == "correct":
    prob_optim_v3.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWCorrect(), promotes=["*"]
    )
else:
    prob_optim_v3.model.add_subsystem(
        name="mtow_solving", subsys=SizingLoopMTOWExercise(), promotes=["*"]
    )

# Add the solver for the MTOW loop and tune them to fit what we did in the MDA
prob_optim_v3.model.nonlinear_solver = om.NonlinearBlockGS()
prob_optim_v3.model.linear_solver = om.DirectSolver()

prob_optim_v3.model.set_solver_print(level=1)

prob_optim_v3.model.nonlinear_solver.options["maxiter"] = 50
prob_optim_v3.model.nonlinear_solver.options["atol"] = 1e-4

# Add the optimization driver (can be copied as is, fine tuning of the solver is for advanced users)
prob_optim_v3.driver = om.ScipyOptimizeDriver()
prob_optim_v3.driver.options["optimizer"] = "COBYLA"
# prob.driver.options['maxiter'] = 100
prob_optim_v3.driver.options["tol"] = 1e-4

# We define the aspect ratio as a design variables and we constrain it
prob_optim_v3.model.add_design_var("aspect_ratio", lower=1.0, upper=20.0)
# We can now add the cruise speed as a design variable (don't forget to give its unit)
prob_optim_v3.model.add_design_var("cruise_speed", units="m/s", lower=5.0, upper=200.0)
prob_optim_v3.model.add_objective("mission_fuel")

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob_optim_v3.model.approx_totals()

# Setting up the problem
prob_optim_v3.setup()

# Initialization of the design var(s) and giving the mtow its initial value. It is important to initialize at a decent
# value or the optimizer might not converge
prob_optim_v3.set_val(name="mtow", val=500.0)
prob_optim_v3.set_val(name="aspect_ratio", val=10.0)
prob_optim_v3.set_val(name="cruise_speed", val=80.0)

# Runnin the optimization
prob_optim_v3.run_driver()

# Printing the output
print("minimum found at")
print("Aspect ratio :", np.round(prob_optim_v3.get_val("aspect_ratio")[0], 4))
print("Cruise speed :", np.round(prob_optim_v3.get_val("cruise_speed", units="m/s")[0], 4), "m/s")

print("minimum objective")
print(float(np.round(prob_optim_v3.get_val("mission_fuel", units="kg"), 4)), "kg")

The results are very close compared to the previous optimization. The slight difference is due to the fact that, when optimizing "by hand", the combinations are limited by the number of value that we allowed the **AR** and cruise speed to take. Here, thanks to OpenMDAO's capabilities, we can find the optimum solution. Indeed, hadn't we rounded the results, we would have seen that the result procided by OpenMDAO is better by 0.01 kg. The same remark could have been made on the [scipy](03_scipy.ipynb) notebook, had we done the exercise of doing a two variables optimization.

## Partial derivatives

Another feature of the OpenMDAO framework is the possibility to define explicit expression for [partial derivatives](https://openmdao.org/newdocs/versions/latest/advanced_user_guide/advanced_user_guide.html). This will not be detailled in this version of the tutorial but, in essence, this allows to reduce the number of evaluations of the components and thus, considerably reduce computation time.

## Conclusion

As we saw in this notebook, OpenMDAO allows for a much simpler definition of a MDA/MDO problem. The only thing that was really needed was to know how to construct the base bricks (components, no matter the type) and how to assemble them (the groups). Once this is done, the setup of the problem is pretty much always the same which simplify things a lot. In addition to that, OpenMDAO automatically handles the data processing, so as long as the names remain coherent, we don't have to worry about the wrong data being passed along, no matter how much there are. The unit conversion is even carried out spontaneously by OpenMDAO, which is one less source of possible error. 

Besides, the difference in syntax from an MDA to an MDO is very thin and there is no need to change the models regardless of what variable we want to use as design parameter. For reminder, this was a strong drawback identified in the previous notebooks.

There is however a couple of improvement that could be implemented to ease the use of OpenMDAO. The main one is user-friendliness. Indeed, from the creation of the problem and the addition of the module to the definition of the problem inputs, everything is done inside the Python file where all the commands are run. This might be a bit off-putting for new user. FAST-OAD, as will be explained in the [next notebook](05_FAST-OAD.ipynb), uses specific files to handle the setup and data initialization of the problem.

## Bibliography

[1] Raymer, Daniel P. "Aircraft design: a conceptual approach (AIAA Education Series)." Reston, Virginia (2012).